In [19]:
# Load packages and Files

from pyhipp.io import h5
import numpy as np
import argparse
import h5py
from pathlib import Path
import json

from pyhipp_sims.sims.sim_info import SimInfo
from scipy.fft import rfftn, irfftn

import os
import time
from multiprocessing import Pool, cpu_count
from functools import partial

print("Package loaded successfully.")

BASE_PATH = Path('/Volumes/T7/TNG100-1-Dark')
OUTPUT_PATH = Path('/Volumes/T7/halo_web')
CONFIG_FILE = OUTPUT_PATH / 'tng100-1-dark.json'
PREPROCESSED_FILE = OUTPUT_PATH / 'tng100-1-dark_postprocessed.hdf5'

N_grids = 256

num_processes = 8

data = h5py.File(PREPROCESSED_FILE, "r", locking=False)
if data:
    print("Raw data file opened successfully.")

Package loaded successfully.
Raw data file opened successfully.


In [ ]:
# calc eigenvalue
grid_size = 256
data_type = '>f4'  # big-endian float32
input_dir = OUTPUT_PATH / 'tidal_output'
file_prefix = 'tidalfield_sample'
output_filename = OUTPUT_PATH / 'tidal_output/eigenvalues.npy'

components = {
    'Txx': os.path.join(input_dir, f'{file_prefix}_0_0'),
    'Txy': os.path.join(input_dir, f'{file_prefix}_0_1'),
    'Txz': os.path.join(input_dir, f'{file_prefix}_0_2'),
    'Tyy': os.path.join(input_dir, f'{file_prefix}_1_1'),
    'Tyz': os.path.join(input_dir, f'{file_prefix}_1_2'),
    'Tzz': os.path.join(input_dir, f'{file_prefix}_2_2'),
}

tensor_parts = {}
for name, path in components.items():
    if not os.path.exists(path):
        raise FileNotFoundError(f"Error: Cannot find file: {path}")
    data = np.fromfile(path, dtype=data_type).reshape((grid_size, grid_size, grid_size))
    tensor_parts[name] = data
    print(f"Loaded {name}: shape {data.shape}, dtype {data.dtype}")

tensor_field = np.zeros((grid_size, grid_size, grid_size, 3, 3), dtype=np.float32)

tensor_field[..., 0, 0] = tensor_parts['Txx'].astype(np.float32)
tensor_field[..., 1, 1] = tensor_parts['Tyy'].astype(np.float32)
tensor_field[..., 2, 2] = tensor_parts['Tzz'].astype(np.float32)

tensor_field[..., 0, 1] = tensor_parts['Txy'].astype(np.float32)
tensor_field[..., 1, 0] = tensor_parts['Txy'].astype(np.float32)  

tensor_field[..., 0, 2] = tensor_parts['Txz'].astype(np.float32)
tensor_field[..., 2, 0] = tensor_parts['Txz'].astype(np.float32)  
tensor_field[..., 1, 2] = tensor_parts['Tyz'].astype(np.float32)
tensor_field[..., 2, 1] = tensor_parts['Tyz'].astype(np.float32)  

print("Tensor field constructed successfully.")
print(f"Tensor field shape: {tensor_field.shape}")
print(f"Tensor field dtype: {tensor_field.dtype}")


print('Calculating eigenvalues...')
eigenvalues = np.linalg.eigvalsh(tensor_field)
print(f"Shape: {eigenvalues.shape}") 


print(f"\n Save to: {output_filename}")
np.save(output_filename, eigenvalues)

lams = np.load(output_filename)
print(lams.shape)

Loaded Txx: shape (256, 256, 256), dtype >f4
Loaded Txy: shape (256, 256, 256), dtype >f4
Loaded Txz: shape (256, 256, 256), dtype >f4
Loaded Tyy: shape (256, 256, 256), dtype >f4
Loaded Tyz: shape (256, 256, 256), dtype >f4
Loaded Tzz: shape (256, 256, 256), dtype >f4
Tensor field constructed successfully.
Tensor field shape: (256, 256, 256, 3, 3)
Tensor field dtype: float32
Calculating eigenvalues...
Shape: (256, 256, 256, 3)

 Save to: /Volumes/T7/halo_web/tidal_output/eigenvalues.npy
(256, 256, 256, 3)


In [35]:
density_field_filename = OUTPUT_PATH / 'density_field.hdf5'
eigenvalues_filename = OUTPUT_PATH /'tidal_output/eigenvalues.npy'
input_dir = OUTPUT_PATH / 'tidal_output/'
output_hdf5_file = OUTPUT_PATH / 'cf_field_sample.hdf5'
n_grids = 256
l_box = 75.0 

data_type = '>f4'

density_filepath = os.path.join(density_field_filename)
if os.path.exists(density_filepath):
    with h5py.File(density_filepath, 'r') as f:
        print(f.keys())
        delta_sm_x = f['data'][:]
    print(" Successfully read density field.")
else:
    print(f"Error: Cannot find density field file {density_filepath}")
    delta_sm_x = None


eigenvalues_filepath = os.path.join(input_dir, eigenvalues_filename)
if os.path.exists(eigenvalues_filepath):
    lams = np.load(eigenvalues_filepath)
    print(" -> Successfully read eigenvalues.")
else:
    print(f"Error: Cannot find eigenvalues file {eigenvalues_filepath}")
    lams = None

print(f"Creating and writing to HDF5 file: {output_hdf5_file}")

with h5py.File(output_hdf5_file, 'w') as hf:
    if delta_sm_x is not None:
        hf.create_dataset('delta_sm_x', data=delta_sm_x)
        print(" -> 'delta_sm_x'")

    if lams is not None:
        hf.create_dataset('lams', data=lams)
        print(" ->'lams'")

    hf.create_dataset('l_box', data=l_box)
    print(" ->'l_box'")
    hf.create_dataset('n_grids', data=n_grids)
    print(" ->'n_grids'")

print("\nProcessing complete.")

<KeysViewHDF5 ['data', 'l_box', 'n_grids']>
 Successfully read density field.
 -> Successfully read eigenvalues.
Creating and writing to HDF5 file: /Volumes/T7/halo_web/cf_field_sample.hdf5
 -> 'delta_sm_x'
 ->'lams'
 ->'l_box'
 ->'n_grids'

Processing complete.


In [ ]:
from pyhipp.field import cubic_box

# generate tidal field
path = OUTPUT_PATH / 'density_field.hdf5'
with h5.File(path) as f:
    rho_x = cubic_box.DensityField.load(f)

tf = cubic_box.TidalField(n_workers=8, r_sm=2.0)
res = tf.run(rho_x)

path = OUTPUT_PATH / 'pyhipp_field_sample.hdf5'
with h5py.File(path, 'w') as f:
    res.dump(f)